<a href="https://colab.research.google.com/github/AnganaB/Sentiment-Analysis-/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly
import tensorflow as tf

     |████████████████████████████████| 522.4MB 29kB/s 
     |████████████████████████████████| 3.0MB 47.8MB/s 
     |████████████████████████████████| 460kB 49.9MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
  Found existing installation: Keras-Preprocessing 1.1.0
    Uninstalling Keras-Preprocessing-1.1.0:
      Successfully uninstalled Keras-Preprocessing-1.1.0


In [0]:
from tensorflow import keras
from tensorflow.keras import layers 

import tensorflow_datasets as tfds
tfds.disable_progress_bar() 

In [3]:
embedding_layer = layers.Embedding(1000, 5)
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
print(result.shape)
result.numpy()

(2, 3, 5)


array([[[ 0.02698601,  0.02004528,  0.01985308,  0.01964689,
         -0.04679755],
        [-0.02021384, -0.01363242, -0.02185481, -0.01683718,
         -0.01360739],
        [ 0.02763114,  0.00622646, -0.00568942, -0.02468425,
         -0.00262558]],

       [[ 0.04031687,  0.00602959,  0.03000039,  0.04100487,
          0.00411282],
        [ 0.01987252, -0.04768006,  0.01122787, -0.01153723,
          0.03862769],
        [ 0.04800132,  0.01668774, -0.00170603, -0.0330503 ,
         -0.00612079]]], dtype=float32)

In [4]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteR57MZC/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteR57MZC/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteR57MZC/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [6]:
encoder = info.features['text'].encoder # "_" represents spaces in the vocabulary
print('Vocabulary size: {}'.format(encoder.vocab_size))
encoder.subwords[:10]

Vocabulary size: 8185


['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br']

In [7]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [8]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [0]:
# used to standardize the lengths of reviews
train_batches = train_data.shuffle(10000).padded_batch(64)
test_batches = test_data.shuffle(10000).padded_batch(64) 

In [10]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[  12, 7422, 2054, ...,    0,    0,    0],
       [2882,  160,  270, ...,    0,    0,    0],
       [ 394, 4136, 7961, ...,    0,    0,    0],
       ...,
       [  19,   98,    5, ...,    0,    0,    0],
       [  62,   27,  180, ...,    0,    0,    0],
       [  62,  435,    9, ...,    0,    0,    0]])

In [11]:
embedding_dim=64

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, embedding_dim, mask_zero = True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
__________________________________________________

In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

Epoch 1/10
391/391 [==============================] - 1657s 4s/step - loss: 0.6330 - accuracy: 0.5902 - val_loss: 0.4437 - val_accuracy: 0.8083
Epoch 2/10
391/391 [==============================] - 1696s 4s/step - loss: 0.3576 - accuracy: 0.8536 - val_loss: 0.3318 - val_accuracy: 0.8474
Epoch 3/10
391/391 [==============================] - 1705s 4s/step - loss: 0.2514 - accuracy: 0.9091 - val_loss: 0.3152 - val_accuracy: 0.8641
Epoch 4/10
391/391 [==============================] - 1648s 4s/step - loss: 0.2062 - accuracy: 0.9285 - val_loss: 0.3699 - val_accuracy: 0.8609
Epoch 5/10
391/391 [==============================] - 1634s 4s/step - loss: 0.1724 - accuracy: 0.9437 - val_loss: 0.3267 - val_accuracy: 0.8703
Epoch 6/10
391/391 [==============================] - 1628s 4s/step - loss: 0.1467 - accuracy: 0.9547 - val_loss: 0.3417 - val_accuracy: 0.8719
Epoch 7/10
391/391 [==============================] - 1681s 4s/step - loss: 0.1251 - accuracy: 0.9639 - val_loss: 0.3941 - val_accuracy:

In [15]:
test_loss, test_acc = model.evaluate(test_batches)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

391/391 [==============================] - 353s 902ms/step - loss: 0.4830 - accuracy: 0.8517
Test Loss: 0.4830092191696167
Test Accuracy: 0.8517199754714966


In [0]:
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sample_pred_text, pad):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)

  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))

  return (predictions)

In [25]:
sample_pred_text = ('The movie was good. The animation and the graphics '
                    'were awesome. I would definitely recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print(predictions)  # if prediction > 0.5, positive review

[[0.8087221]]
